# EigenSpace Mapping: 53-TET Chord Positions

This notebook maps **53-TET (53 equal temperament)** chords to the **EigenSpace** coordinate system.

## EigenSpace Coordinate System
- **(α, β, γ)** = frequency ratios of the 2nd, 3rd, and 4th notes relative to the root
- **Constraint**: α ≤ β ≤ γ (tetrahedron region, eliminates redundant permutations)
- **Range**: 1.0 to 2.0 (one octave above root)

## 53-TET System
- 53 equal divisions of the octave
- Step size = 2^(1/53) ≈ 1.01316 (≈ 22.64 cents)
- Closely approximates just intonation intervals
- Superior to 12-TET for harmonic accuracy

## Goals
1. Define all 53-TET chord positions in EigenSpace
2. Find local minima (harmonic nodes) in the dissonance map  
3. Compare 53-TET chords with 12-TET positions
4. Create reference data for microtonal augmentation pipeline

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.ndimage import minimum_filter, maximum_filter
import os
from typing import List, Tuple, Dict, Optional
import json

# ============================================================================
# 53-TET FUNDAMENTAL FUNCTIONS
# ============================================================================

def get_53tet_ratio(steps: int) -> float:
    """
    Convert 53-TET steps to frequency ratio.
    
    53-TET divides the octave (2:1) into 53 equal parts.
    Each step = 2^(1/53) ≈ 1.01316 ratio (≈ 22.64 cents)
    
    Args:
        steps: Number of 53-TET steps from root (0-53)
    
    Returns:
        Frequency ratio relative to root
    """
    return 2.0 ** (steps / 53.0)


def get_12tet_ratio(semitones: int) -> float:
    """
    Convert 12-TET semitones to frequency ratio.
    
    Args:
        semitones: Number of semitones from root (0-12)
    
    Returns:
        Frequency ratio relative to root
    """
    return 2.0 ** (semitones / 12.0)


# Helper shorthand for ratio conversion
r = get_53tet_ratio
r12 = get_12tet_ratio

print("53-TET step examples:")
print(f"  Major third (18 steps):  {r(18):.6f}  (vs JI 5/4 = {5/4:.6f})")
print(f"  Perfect fifth (31 steps): {r(31):.6f}  (vs JI 3/2 = {3/2:.6f})")
print(f"  Major seventh (49 steps): {r(49):.6f}  (vs JI 15/8 = {15/8:.6f})")

In [2]:
# ============================================================================
# 53-TET INTERVAL DEFINITIONS
# ============================================================================
# These match exactly the JavaScript definitions from Harmonic_Eigenspace.js

# THIRDS (from root) - 53-TET step values
THIRDS_53 = {
    'sm': 11,   # super-minor (wolf third)
    'vm': 12,   # sub-minor
    'm':  13,   # minor third (300 cents in 12-TET → 13.25 steps)
    '^m': 14,   # super-minor / neutral low
    'n':  15,   # neutral third
    'N':  16,   # neutral third high
    'vM': 17,   # sub-major
    'M':  18,   # major third (400 cents → 17.67 steps, rounds to 18)
    '^M': 19,   # super-major
    'SM': 20,   # wide major (septimal)
}

# FIFTHS (from root) - 53-TET step values  
FIFTHS_53 = {
    'subdim': 29,  # sub-diminished
    'dim':    30,  # diminished fifth
    'vP':     30,  # sub-perfect (same as dim)
    'P':      31,  # perfect fifth (700 cents → 30.92 steps)
    '^P':     32,  # super-perfect
    'aug':    32,  # augmented fifth (same as ^P)
    'upaug':  33,  # up-augmented
}

# SEVENTHS (from root) - 53-TET step values
SEVENTHS_53 = {
    'sm': 42,  # super-diminished 7th
    'vm': 43,  # sub-minor 7th
    'm':  44,  # minor seventh (1000 cents → 44.17 steps)
    '^m': 45,  # super-minor 7th
    'n':  46,  # neutral 7th
    'N':  47,  # neutral 7th high
    'vM': 48,  # sub-major 7th
    'M':  49,  # major seventh (1100 cents → 48.58 steps)
    '^M': 50,  # super-major 7th
    'SM': 51,  # wide major 7th
}

print("53-TET Interval Mappings:")
print(f"\nThirds (steps → ratio → cents):")
for name, steps in THIRDS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>3}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

print(f"\nFifths:")
for name, steps in FIFTHS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>6}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

print(f"\nSevenths:")
for name, steps in SEVENTHS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>3}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

In [3]:
# ============================================================================
# 53-TET CHORD DEFINITIONS - EXACT PORT FROM Harmonic_Eigenspace.js
# ============================================================================

def get_53tet_chord_positions() -> List[Tuple[str, float, float, float]]:
    """
    Get all 53-TET chord positions in EigenSpace (α, β, γ) coordinates.
    
    DIRECT PORT from get53TETChordPositions() in Harmonic_Eigenspace.js
    
    Structure: Each chord is [name, r(third), r(fifth), r(seventh)]
    - Organized by third type (SM=20 down to sm=11)
    - Each third has 10 seventh variants
    - Most use perfect fifth (31), half-dim uses 26
    
    Returns:
        List of (name, α, β, γ) tuples
    """
    chords = []
    
    # ===== SUPER-MAJOR THIRD (SM=20) COMBINATIONS =====
    chords.extend([
        ("SMSM7",   r(20), r(31), r(51)),
        ("SM^M7",   r(20), r(31), r(50)),
        ("SMmaj7",  r(20), r(31), r(49)),
        ("SMvM7",   r(20), r(31), r(48)),
        ("SMN7",    r(20), r(31), r(47)),
        ("SMn7",    r(20), r(31), r(46)),
        ("SM^m7",   r(20), r(31), r(45)),
        ("SMm7",    r(20), r(31), r(44)),
        ("SMvm7",   r(20), r(31), r(43)),
        ("SMsm7",   r(20), r(31), r(42)),
    ])
    
    # ===== UP-MAJOR THIRD (^M=19) COMBINATIONS =====
    chords.extend([
        ("^MSM7",   r(19), r(31), r(51)),
        ("^M^M7",   r(19), r(31), r(50)),
        ("^Mmaj7",  r(19), r(31), r(49)),
        ("^MvM7",   r(19), r(31), r(48)),
        ("^MN7",    r(19), r(31), r(47)),
        ("^Mn7",    r(19), r(31), r(46)),
        ("^M^m7",   r(19), r(31), r(45)),
        ("^Mm7",    r(19), r(31), r(44)),
        ("^Mvm7",   r(19), r(31), r(43)),
        ("^Msm7",   r(19), r(31), r(42)),
    ])
    
    # ===== MAJOR THIRD (M=18) COMBINATIONS =====
    chords.extend([
        ("MSM7",    r(18), r(31), r(51)),
        ("M^M7",    r(18), r(31), r(50)),
        ("maj7",    r(18), r(31), r(49)),
        ("MvM7",    r(18), r(31), r(48)),
        ("MN7",     r(18), r(31), r(47)),
        ("Mn7",     r(18), r(31), r(46)),
        ("M^m7",    r(18), r(31), r(45)),
        ("Mm7",     r(18), r(31), r(44)),
        ("Mvm7",    r(18), r(31), r(43)),
        ("Msm7",    r(18), r(31), r(42)),
    ])
    
    # ===== DOWN-MAJOR THIRD (vM=17) COMBINATIONS =====
    chords.extend([
        ("vMSM7",   r(17), r(31), r(51)),
        ("vM^M7",   r(17), r(31), r(50)),
        ("vMmaj7",  r(17), r(31), r(49)),
        ("vMvM7",   r(17), r(31), r(48)),
        ("vMN7",    r(17), r(31), r(47)),
        ("vMn7",    r(17), r(31), r(46)),
        ("vM^m7",   r(17), r(31), r(45)),
        ("vMm7",    r(17), r(31), r(44)),
        ("vMvm7",   r(17), r(31), r(43)),
        ("vMsm7",   r(17), r(31), r(42)),
    ])
    
    # ===== NEUTRAL-MAJOR THIRD (N=16) COMBINATIONS =====
    chords.extend([
        ("NSM7",    r(16), r(31), r(51)),
        ("N^M7",    r(16), r(31), r(50)),
        ("Nmaj7",   r(16), r(31), r(49)),
        ("NvM7",    r(16), r(31), r(48)),
        ("NN7",     r(16), r(31), r(47)),
        ("Nn7",     r(16), r(31), r(46)),
        ("N^m7",    r(16), r(31), r(45)),
        ("Nm7",     r(16), r(31), r(44)),
        ("Nvm7",    r(16), r(31), r(43)),
        ("Nsm7",    r(16), r(31), r(42)),
    ])
    
    # ===== NEUTRAL-MINOR THIRD (n=15) COMBINATIONS =====
    chords.extend([
        ("nSM7",    r(15), r(31), r(51)),
        ("n^M7",    r(15), r(31), r(50)),
        ("nmaj7",   r(15), r(31), r(49)),
        ("nvM7",    r(15), r(31), r(48)),
        ("nN7",     r(15), r(31), r(47)),
        ("nn7",     r(15), r(31), r(46)),
        ("n^m7",    r(15), r(31), r(45)),
        ("nm7",     r(15), r(31), r(44)),
        ("nvm7",    r(15), r(31), r(43)),
        ("nsm7",    r(15), r(31), r(42)),
    ])
    
    # ===== UP-MINOR THIRD (^m=14) COMBINATIONS =====
    chords.extend([
        ("^mSM7",   r(14), r(31), r(51)),
        ("^m^M7",   r(14), r(31), r(50)),
        ("^mmaj7",  r(14), r(31), r(49)),
        ("^mvM7",   r(14), r(31), r(48)),
        ("^mN7",    r(14), r(31), r(47)),
        ("^mn7",    r(14), r(31), r(46)),
        ("^m^m7",   r(14), r(31), r(45)),
        ("^mm7",    r(14), r(31), r(44)),
        ("^mvm7",   r(14), r(31), r(43)),
        ("^msm7",   r(14), r(31), r(42)),
    ])
    
    # ===== MINOR THIRD (m=13) COMBINATIONS =====
    chords.extend([
        ("mSM7",    r(13), r(31), r(51)),
        ("m^M7",    r(13), r(31), r(50)),
        ("mmaj7",   r(13), r(31), r(49)),
        ("mvM7",    r(13), r(31), r(48)),
        ("mN7",     r(13), r(31), r(47)),
        ("mn7",     r(13), r(31), r(46)),
        ("m^m7",    r(13), r(31), r(45)),
        ("m7",      r(13), r(31), r(44)),
        ("mvm7",    r(13), r(31), r(43)),
        ("msm7",    r(13), r(31), r(42)),
    ])
    
    # ===== DOWN-MINOR THIRD (vm=12) COMBINATIONS =====
    chords.extend([
        ("vmSM7",   r(12), r(31), r(51)),
        ("vm^M7",   r(12), r(31), r(50)),
        ("vmmaj7",  r(12), r(31), r(49)),
        ("vmvM7",   r(12), r(31), r(48)),
        ("vmN7",    r(12), r(31), r(47)),
        ("vmn7",    r(12), r(31), r(46)),
        ("vm^m7",   r(12), r(31), r(45)),
        ("vm7",     r(12), r(31), r(44)),
        ("vmvm7",   r(12), r(31), r(43)),
        ("vmsm7",   r(12), r(31), r(42)),
    ])
    
    # ===== SUB-MINOR THIRD (sm=11) COMBINATIONS =====
    chords.extend([
        ("smSM7",   r(11), r(31), r(51)),
        ("sm^M7",   r(11), r(31), r(50)),
        ("smmaj7",  r(11), r(31), r(49)),
        ("smvM7",   r(11), r(31), r(48)),
        ("smN7",    r(11), r(31), r(47)),
        ("smn7",    r(11), r(31), r(46)),
        ("sm^m7",   r(11), r(31), r(45)),
        ("sm7",     r(11), r(31), r(44)),
        ("smvm7",   r(11), r(31), r(43)),
        ("smsm7",   r(11), r(31), r(42)),
    ])
    
    # ===== HALF-DIMINISHED UP-MINOR (^m=14, fifth=26) =====
    chords.extend([
        ("øSM7",    r(14), r(26), r(51)),
        ("ø^M7",    r(14), r(26), r(50)),
        ("ømaj7",   r(14), r(26), r(49)),
        ("øvM7",    r(14), r(26), r(48)),
        ("øN7",     r(14), r(26), r(47)),
        ("øn7",     r(14), r(26), r(46)),
        ("ø^m7",    r(14), r(26), r(45)),
        ("ø7",      r(14), r(26), r(44)),
        ("øvm7",    r(14), r(26), r(43)),
        ("øsm7",    r(14), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED MINOR (m=13, fifth=26) =====
    chords.extend([
        ("øS7",     r(13), r(26), r(51)),
        ("ø^M7-m",  r(13), r(26), r(50)),
        ("ømaj7-m", r(13), r(26), r(49)),
        ("øvM7-m",  r(13), r(26), r(48)),
        ("øNM7",    r(13), r(26), r(47)),
        ("øn7-m",   r(13), r(26), r(46)),
        ("øv7",     r(13), r(26), r(45)),
        ("ø7-m",    r(13), r(26), r(44)),
        ("øvm7-m",  r(13), r(26), r(43)),
        ("øsm7-m",  r(13), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED DOWN-MINOR (vm=12, fifth=26) =====
    chords.extend([
        ("vøS7",    r(12), r(26), r(51)),
        ("vø^M7",   r(12), r(26), r(50)),
        ("vømaj7",  r(12), r(26), r(49)),
        ("vøvM7",   r(12), r(26), r(48)),
        ("vøN7",    r(12), r(26), r(47)),
        ("vøn7",    r(12), r(26), r(46)),
        ("vø^m7",   r(12), r(26), r(45)),
        ("vø7",     r(12), r(26), r(44)),
        ("vøvm7",   r(12), r(26), r(43)),
        ("vøsm7",   r(12), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED SUB-MINOR (sm=11, fifth=26) =====
    chords.extend([
        ("søS7",    r(11), r(26), r(51)),
        ("sø^M7",   r(11), r(26), r(50)),
        ("sømaj7",  r(11), r(26), r(49)),
        ("søvM7",   r(11), r(26), r(48)),
        ("søN7",    r(11), r(26), r(47)),
        ("søn7",    r(11), r(26), r(46)),
        ("sø^m7",   r(11), r(26), r(45)),
        ("sø7",     r(11), r(26), r(44)),
        ("søvm7",   r(11), r(26), r(43)),
        ("søsm7",   r(11), r(26), r(42)),
    ])
    
    # ===== AUGMENTED (M=18, fifth=35) =====
    chords.extend([
        ("M+S7",    r(18), r(35), r(51)),
        ("M+^M7",   r(18), r(35), r(50)),
        ("M+maj7",  r(18), r(35), r(49)),
        ("M+vM7",   r(18), r(35), r(48)),
        ("M+NM7",   r(18), r(35), r(47)),
        ("M+N7",    r(18), r(35), r(46)),
        ("M+n7",    r(18), r(35), r(45)),
        ("M+m7",    r(18), r(35), r(44)),
        ("M+vm7",   r(18), r(35), r(43)),
        ("M+sm7",   r(18), r(35), r(42)),
    ])
    
    # ===== SUSPENDED =====
    chords.extend([
        ("7sus4",   r(22), r(31), r(44)),
        ("sus2",    r(9),  r(22), r(31)),
    ])
    
    return chords


# Generate the chord list
chords_53tet = get_53tet_chord_positions()
print(f"Total 53-TET chords defined: {len(chords_53tet)}")
print("\nFirst 15 chords (Super-Major third family):")
for name, alpha, beta, gamma in chords_53tet[:15]:
    print(f"  {name:12s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")
print("\n...")
print("\nHalf-diminished family (ø7 variants):")
for name, alpha, beta, gamma in chords_53tet[100:110]:
    print(f"  {name:12s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")

In [4]:
# ============================================================================
# 12-TET CHORD DEFINITIONS - FROM Harmonic_Eigenspace.js get12TETChordPositions()
# ============================================================================

def get_12tet_chord_positions() -> List[Tuple[str, float, float, float]]:
    """
    Get standard 12-TET chord positions - direct port from JS.
    
    Returns:
        List of (name, α, β, γ) tuples
    """
    return [
        ("maj7",    r12(4),  r12(7),  r12(11)),
        ("min7",    r12(3),  r12(7),  r12(10)),
        ("dom7",    r12(4),  r12(7),  r12(10)),
        ("mM7",     r12(3),  r12(7),  r12(11)),
        ("ø7",      r12(3),  r12(6),  r12(10)),
        ("dim7",    r12(3),  r12(6),  r12(9)),
        ("aug7",    r12(4),  r12(8),  r12(10)),
        ("augM7",   r12(4),  r12(8),  r12(11)),
        ("7sus4",   r12(5),  r12(7),  r12(10)),
        ("sus2",    r12(2),  r12(5),  r12(7)),
    ]


chords_12tet = get_12tet_chord_positions()
print(f"Total 12-TET reference chords: {len(chords_12tet)}")
print("\n12-TET chord positions:")
for name, alpha, beta, gamma in chords_12tet:
    print(f"  {name:10s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")

## Load Pre-computed Dissonance Map

The EigenSpace dissonance map was pre-computed at 220 Hz with:
- 150³ = 3,375,000 points  
- α, β, γ range: 1.0 to 2.0
- Plomp-Levelt dissonance model with 6 harmonics

In [5]:
# ============================================================================
# LOAD PRE-COMPUTED DISSONANCE MAP
# ============================================================================

def load_precomputed_dissonance_map(
    base_freq: int = 220,
    n_points: int = 150,
    r_low: float = 1.0,
    r_high: float = 2.0,
    dataset_path: str = "../dataset/EigenSpace_Data"
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Load pre-computed dissonance map from binary chunk files.
    
    Args:
        base_freq: Base frequency the map was computed at (Hz)
        n_points: Resolution (n³ grid points)
        r_low, r_high: Ratio range
        dataset_path: Path to EigenSpace_Data folder
        
    Returns:
        (alpha_range, beta_range, gamma_range, dissonance_3d)
    """
    print(f"Loading {n_points}³ pre-computed dataset from {dataset_path}...")
    
    # Find all chunk files
    chunk_files = sorted([
        f for f in os.listdir(dataset_path)
        if f.startswith(f"harmonic-{base_freq}Hz-{n_points}nodes-chunk")
    ])
    
    if not chunk_files:
        raise FileNotFoundError(
            f"No dataset chunks found for {base_freq}Hz, {n_points} nodes"
        )
    
    # Load all chunks
    all_data = []
    for chunk_file in chunk_files:
        chunk_path = os.path.join(dataset_path, chunk_file)
        chunk_data = np.fromfile(chunk_path, dtype=np.float32)
        all_data.append(chunk_data)
        print(f"  ✓ {chunk_file}: {len(chunk_data):,} values")
    
    # Concatenate and reshape
    flat_data = np.concatenate(all_data)
    dissonance_3d = flat_data.reshape((n_points, n_points, n_points))
    
    # Recreate coordinate ranges
    alpha_range = np.linspace(r_low, r_high, n_points)
    beta_range = np.linspace(r_low, r_high, n_points)
    gamma_range = np.linspace(r_low, r_high, n_points)
    
    print(f"\n✓ Loaded shape: {dissonance_3d.shape}")
    print(f"✓ Dissonance range: {dissonance_3d.min():.4f} to {dissonance_3d.max():.4f}")
    
    return alpha_range, beta_range, gamma_range, dissonance_3d


# Load the dataset
alpha_range, beta_range, gamma_range, dissonance_3d = load_precomputed_dissonance_map(
    base_freq=220,
    n_points=150
)

In [6]:
# ============================================================================
# DISSONANCE LOOKUP FUNCTIONS
# ============================================================================

def get_dissonance_at_point(
    alpha: float, beta: float, gamma: float,
    alpha_range: np.ndarray, beta_range: np.ndarray, gamma_range: np.ndarray,
    dissonance_3d: np.ndarray
) -> Optional[float]:
    """
    Get dissonance value at a specific (α, β, γ) coordinate using trilinear interpolation.
    
    Args:
        alpha, beta, gamma: Target coordinates
        alpha_range, beta_range, gamma_range: Coordinate arrays
        dissonance_3d: 3D dissonance map
        
    Returns:
        Interpolated dissonance value, or None if out of bounds
    """
    # Check bounds
    if (alpha < alpha_range[0] or alpha > alpha_range[-1] or
        beta < beta_range[0] or beta > beta_range[-1] or
        gamma < gamma_range[0] or gamma > gamma_range[-1]):
        return None
    
    # Find indices (for trilinear interpolation)
    def find_bracket(val, arr):
        idx = np.searchsorted(arr, val) - 1
        idx = max(0, min(idx, len(arr) - 2))
        t = (val - arr[idx]) / (arr[idx + 1] - arr[idx])
        return idx, t
    
    i, ti = find_bracket(alpha, alpha_range)
    j, tj = find_bracket(beta, beta_range)
    k, tk = find_bracket(gamma, gamma_range)
    
    # Trilinear interpolation
    c000 = dissonance_3d[i, j, k]
    c100 = dissonance_3d[i+1, j, k]
    c010 = dissonance_3d[i, j+1, k]
    c110 = dissonance_3d[i+1, j+1, k]
    c001 = dissonance_3d[i, j, k+1]
    c101 = dissonance_3d[i+1, j, k+1]
    c011 = dissonance_3d[i, j+1, k+1]
    c111 = dissonance_3d[i+1, j+1, k+1]
    
    # Interpolate along x (alpha)
    c00 = c000 * (1 - ti) + c100 * ti
    c10 = c010 * (1 - ti) + c110 * ti
    c01 = c001 * (1 - ti) + c101 * ti
    c11 = c011 * (1 - ti) + c111 * ti
    
    # Interpolate along y (beta)
    c0 = c00 * (1 - tj) + c10 * tj
    c1 = c01 * (1 - tj) + c11 * tj
    
    # Interpolate along z (gamma)
    return c0 * (1 - tk) + c1 * tk


# Test with a known chord
test_chord = chords_53tet[0]  # sm_min7
diss = get_dissonance_at_point(
    test_chord[1], test_chord[2], test_chord[3],
    alpha_range, beta_range, gamma_range, dissonance_3d
)
print(f"Test: {test_chord[0]} dissonance = {diss:.4f}")

## Find Local Minima (Harmonic Nodes)

Local minima in the dissonance map represent **natural consonant points** - 
places where all four notes of a tetrachord interact with minimal beating/roughness.

These are the "harmonic nodes" that serve as reference points for microtonal navigation.

In [7]:
# ============================================================================
# FIND HARMONIC NODES (Local Minima in Dissonance Field)
# ============================================================================
# Exact port from Harmonic_Eigenspace.js findHarmonicNodes()
# Computes once and saves to file for reuse

HARMONIC_NODES_FILE = "../dataset/EigenSpace_Data/harmonic_nodes.json"

def find_harmonic_nodes(
    alpha_range: np.ndarray,
    beta_range: np.ndarray,
    gamma_range: np.ndarray,
    dissonance_3d: np.ndarray,
    num_nodes: int = 50,
    filter_size: int = 5
) -> List[Dict]:
    """
    Find local minima in the dissonance field.
    Exact port of findHarmonicNodes() from Harmonic_Eigenspace.js
    """
    nodes = []
    step_size = (alpha_range[-1] - alpha_range[0]) / len(alpha_range)
    boundary_margin = max(3, int(0.1 * len(alpha_range)))
    prominence_radius = max(6, filter_size * 2)
    
    n_alpha = len(alpha_range)
    n_beta = len(beta_range)
    n_gamma = len(gamma_range)
    
    for i in range(boundary_margin, n_alpha - boundary_margin):
        for j in range(boundary_margin, n_beta - boundary_margin):
            for k in range(boundary_margin, n_gamma - boundary_margin):
                value = dissonance_3d[i, j, k]
                if np.isnan(value):
                    continue
                
                alpha_val = alpha_range[i]
                beta_val = beta_range[j]
                gamma_val = gamma_range[k]
                
                # Tetrahedron constraint: α ≤ β ≤ γ
                if alpha_val > beta_val or beta_val > gamma_val:
                    continue
                
                # Spacing check
                if abs(alpha_val - beta_val) < step_size * 2 or abs(beta_val - gamma_val) < step_size * 2:
                    continue
                
                # Check if local minimum
                is_min = True
                for di in range(-filter_size, filter_size + 1):
                    if not is_min:
                        break
                    for dj in range(-filter_size, filter_size + 1):
                        if not is_min:
                            break
                        for dk in range(-filter_size, filter_size + 1):
                            if di == 0 and dj == 0 and dk == 0:
                                continue
                            ni, nj, nk = i + di, j + dj, k + dk
                            if 0 <= ni < n_alpha and 0 <= nj < n_beta and 0 <= nk < n_gamma:
                                neighbor_value = dissonance_3d[ni, nj, nk]
                                # Skip NaN neighbors (invalid constraint violations)
                                if not np.isnan(neighbor_value):
                                    if value >= neighbor_value:
                                        is_min = False
                                        break
                
                if not is_min:
                    continue
                
                # Calculate prominence
                max_in_radius = value
                for di in range(-prominence_radius, prominence_radius + 1):
                    for dj in range(-prominence_radius, prominence_radius + 1):
                        for dk in range(-prominence_radius, prominence_radius + 1):
                            ni, nj, nk = i + di, j + dj, k + dk
                            if 0 <= ni < n_alpha and 0 <= nj < n_beta and 0 <= nk < n_gamma:
                                if not np.isnan(dissonance_3d[ni, nj, nk]):
                                    max_in_radius = max(max_in_radius, dissonance_3d[ni, nj, nk])
                
                prominence = max_in_radius - value
                if prominence < 0.001:
                    continue
                
                # Calculate gradient
                gradient_sum = 0
                gradient_count = 0
                for di in range(-1, 2):
                    for dj in range(-1, 2):
                        for dk in range(-1, 2):
                            if di == 0 and dj == 0 and dk == 0:
                                continue
                            ni, nj, nk = i + di, j + dj, k + dk
                            if 0 <= ni < n_alpha and 0 <= nj < n_beta and 0 <= nk < n_gamma:
                                if not np.isnan(dissonance_3d[ni, nj, nk]):
                                    gradient_sum += abs(dissonance_3d[ni, nj, nk] - value)
                                    gradient_count += 1
                
                avg_gradient = gradient_sum / gradient_count if gradient_count > 0 else 0
                curvature = prominence * (1 + avg_gradient * 10)
                
                nodes.append({
                    'alpha': float(alpha_val),
                    'beta': float(beta_val),
                    'gamma': float(gamma_val),
                    'dissonance': float(value),
                    'prominence': float(prominence),
                    'curvature': float(curvature)
                })
    
    nodes.sort(key=lambda x: x['curvature'], reverse=True)
    return nodes[:num_nodes]


def load_or_compute_harmonic_nodes(
    alpha_range: np.ndarray,
    beta_range: np.ndarray,
    gamma_range: np.ndarray,
    dissonance_3d: np.ndarray,
    num_nodes: int = 50,
    filter_size: int = 5,
    force_recompute: bool = False
) -> List[Dict]:
    """Load harmonic nodes from file if exists, otherwise compute and save."""
    
    if os.path.exists(HARMONIC_NODES_FILE) and not force_recompute:
        print(f"Loading pre-computed harmonic nodes from {HARMONIC_NODES_FILE}...")
        with open(HARMONIC_NODES_FILE, 'r') as f:
            nodes = json.load(f)
        print(f"✓ Loaded {len(nodes)} harmonic nodes")
        return nodes
    
    print("Computing harmonic nodes (this may take a few minutes)...")
    nodes = find_harmonic_nodes(
        alpha_range, beta_range, gamma_range, dissonance_3d,
        num_nodes, filter_size
    )
    
    # Save to file
    with open(HARMONIC_NODES_FILE, 'w') as f:
        json.dump(nodes, f, indent=2)
    print(f"✓ Saved {len(nodes)} harmonic nodes to {HARMONIC_NODES_FILE}")
    
    return nodes


# Load or compute harmonic nodes
harmonic_nodes = load_or_compute_harmonic_nodes(
    alpha_range, beta_range, gamma_range, dissonance_3d,
    num_nodes=77, filter_size=5,
    force_recompute=False  # Set True to recompute
)

print(f"\nTop 15 harmonic nodes:")
for i, node in enumerate(harmonic_nodes[:15]):
    print(f"  {i+1:2d}. α={node['alpha']:.4f}, β={node['beta']:.4f}, γ={node['gamma']:.4f} | D={node['dissonance']:.3f}")

In [8]:
# Debug: check array shape and indexing
print(f"dissonance_3d shape: {dissonance_3d.shape}")
print(f"alpha_range: {len(alpha_range)} values from {alpha_range[0]:.4f} to {alpha_range[-1]:.4f}")
print(f"beta_range: {len(beta_range)} values from {beta_range[0]:.4f} to {beta_range[-1]:.4f}")
print(f"gamma_range: {len(gamma_range)} values from {gamma_range[0]:.4f} to {gamma_range[-1]:.4f}")

# Test indexing: dissonance_3d[i, j, k] should give value at (alpha[i], beta[j], gamma[k])
i, j, k = 50, 75, 100
print(f"\nTest point dissonance_3d[{i}, {j}, {k}] = {dissonance_3d[i, j, k]:.4f}")
print(f"  Should be at α={alpha_range[i]:.4f}, β={beta_range[j]:.4f}, γ={gamma_range[k]:.4f}")

In [9]:
# ============================================================================
# MAP ALL CHORDS WITH DISSONANCE VALUES
# ============================================================================

def map_chords_to_eigenspace(
    chords: List[Tuple[str, float, float, float]],
    alpha_range: np.ndarray,
    beta_range: np.ndarray,
    gamma_range: np.ndarray,
    dissonance_3d: np.ndarray,
    system_name: str = "53-TET"
) -> pd.DataFrame:
    """
    Map chord positions to EigenSpace and calculate their dissonance values.
    
    Args:
        chords: List of (name, α, β, γ) tuples
        alpha_range, beta_range, gamma_range: Coordinate arrays
        dissonance_3d: 3D dissonance map
        system_name: Name of the tuning system
        
    Returns:
        DataFrame with chord positions and dissonance values
    """
    records = []
    
    for name, alpha, beta, gamma in chords:
        # Check if chord is within tetrahedron constraint
        in_tetrahedron = alpha <= beta <= gamma
        
        # Get dissonance value
        diss = get_dissonance_at_point(
            alpha, beta, gamma,
            alpha_range, beta_range, gamma_range, dissonance_3d
        )
        
        # Calculate cents from root for each interval
        alpha_cents = 1200 * np.log2(alpha) if alpha > 0 else 0
        beta_cents = 1200 * np.log2(beta) if beta > 0 else 0
        gamma_cents = 1200 * np.log2(gamma) if gamma > 0 else 0
        
        records.append({
            'name': name,
            'system': system_name,
            'alpha': alpha,
            'beta': beta,
            'gamma': gamma,
            'alpha_cents': alpha_cents,
            'beta_cents': beta_cents,
            'gamma_cents': gamma_cents,
            'dissonance': diss,
            'in_tetrahedron': in_tetrahedron
        })
    
    return pd.DataFrame(records)


# Map both 53-TET and 12-TET chords
df_53tet = map_chords_to_eigenspace(
    chords_53tet, alpha_range, beta_range, gamma_range, dissonance_3d, "53-TET"
)
df_12tet = map_chords_to_eigenspace(
    chords_12tet, alpha_range, beta_range, gamma_range, dissonance_3d, "12-TET"
)

# Combine into single DataFrame
df_all_chords = pd.concat([df_53tet, df_12tet], ignore_index=True)

print(f"✓ Mapped {len(df_53tet)} 53-TET chords")
print(f"✓ Mapped {len(df_12tet)} 12-TET chords")
print(f"✓ Total: {len(df_all_chords)} chords\n")

# Show statistics
print("Dissonance Statistics by System:")
print(df_all_chords.groupby('system')['dissonance'].describe())

In [10]:
# ============================================================================
# COMPARE 53-TET vs 12-TET DISSONANCE
# ============================================================================

# Find matching chord types and compare dissonance
common_chords = ['maj7', 'dom7', 'min7', 'minMaj7', 'dim7', 'hdim7', 'aug7', 'augMaj7']

print("Dissonance Comparison: 53-TET vs 12-TET")
print("=" * 60)
print(f"{'Chord':<12} {'12-TET':<12} {'53-TET':<12} {'Difference':<12} {'Better'}")
print("-" * 60)

for chord_name in common_chords:
    tet12 = df_12tet[df_12tet['name'] == chord_name]
    tet53 = df_53tet[df_53tet['name'] == chord_name]
    
    if len(tet12) > 0 and len(tet53) > 0:
        d12 = tet12['dissonance'].iloc[0]
        d53 = tet53['dissonance'].iloc[0]
        diff = d53 - d12
        better = "53-TET" if d53 < d12 else "12-TET" if d12 < d53 else "Equal"
        
        print(f"{chord_name:<12} {d12:<12.4f} {d53:<12.4f} {diff:<+12.4f} {better}")

print("-" * 60)
print("\nNote: Lower dissonance = more consonant (smoother sound)")

## 3D Visualization: EigenSpace with All Chord Systems

Interactive 3D plot showing:
- **Dissonance map** (colored surface/contours)
- **Local minima** (white dots) - harmonic nodes
- **12-TET chords** (squares) - standard equal temperament
- **53-TET chords** (circles) - microtonal positions

In [11]:
# ============================================================================
# 3D VISUALIZATION (matching dissonance_4D.ipynb) - FIXED
# ============================================================================

def create_eigenspace_visualization_fixed(
    alpha_range: np.ndarray,
    beta_range: np.ndarray,
    gamma_range: np.ndarray,
    dissonance_3d: np.ndarray,
    harmonic_nodes: List[Dict],
    df_chords: pd.DataFrame,
    show_all_layers: bool = True
) -> go.Figure:
    """
    Create interactive 3D visualization of EigenSpace with chord overlays.
    Uses ALL gamma layers for continuous appearance (matching dissonance_4D.ipynb).
    
    CRITICAL FIXES from dissonance_4D.ipynb:
    - surface_data[j, i] = np.nan (NOT [i, j])
    - z=np.full_like(surface_data, gamma_val).T (WITH .T transpose)
    """
    fig = go.Figure()

    # Color range: 5th to 80th percentile
    vmin = np.percentile(dissonance_3d, 5)
    vmax = np.percentile(dissonance_3d, 80)

    # Custom colorscale from dissonance_4D.ipynb
    custom_colorscale = [
        [0.0, "rgba(0, 0, 0, 0.0)"],
        [0.1, "rgba(0, 0, 180, 1.0)"],
        [0.2, "rgba(0, 150, 255, 1.0)"],
        [0.3, "rgba(0, 200, 255, 1.0)"],
        [0.4, "rgba(100, 220, 255, 1.0)"],
        [0.5, "rgba(255, 255, 255, 1.0)"],
        [0.6, "rgba(255, 220, 100, 1.0)"],
        [0.7, "rgba(255, 200, 0, 1.0)"],
        [0.8, "rgba(255, 170, 0, 1.0)"],
        [0.9, "rgba(255, 140, 0, 1.0)"],
        [0.95, "rgba(255, 100, 0, 1.0)"],
        [1.0, "rgba(255, 0, 0, 1.0)"],
    ]

    # Add ALL dissonance layers
    n_levels = len(gamma_range) if show_all_layers else 30
    layer_indices = range(n_levels) if show_all_layers else np.linspace(0, len(gamma_range) - 1, n_levels, dtype=int)

    print(f"Creating {len(list(layer_indices))} layers...")

    for idx in layer_indices:
        if idx % 20 == 0:
            print(f"  Layer {idx}/{len(gamma_range)}")

        gamma_val = gamma_range[idx]
        surface_data = dissonance_3d[:, :, idx].copy()

        # Apply tetrahedron constraint: α ≤ β ≤ γ
        # CRITICAL FIX: [j, i] NOT [i, j]
        for i in range(len(alpha_range)):
            for j in range(len(beta_range)):
                alpha_val = alpha_range[i]
                beta_val = beta_range[j]
                if alpha_val > beta_val or beta_val > gamma_val:
                    surface_data[j, i] = np.nan  # FIXED: [j, i]

        is_first_layer = (idx == 0)
        fig.add_trace(go.Surface(
            x=alpha_range,
            y=beta_range,
            z=np.full_like(surface_data, gamma_val).T,  # FIXED: .T
            surfacecolor=surface_data,
            colorscale=custom_colorscale,
            cmin=vmin,
            cmax=vmax,
            showscale=is_first_layer,
            opacity=0.1,
            connectgaps=False,
            hovertemplate='α=%{x:.4f}<br>β=%{y:.4f}<br>γ=%{z:.4f}<extra></extra>',
            colorbar=dict(
                title=dict(text='Dissonance', font=dict(size=12, color='black')),
                thickness=20,
                len=0.65,
                x=1.0,
                tickfont=dict(size=10, color='black'),
                tickformat='.1f'
            ) if is_first_layer else None
        ))

    # Add harmonic nodes (local minima)
    if harmonic_nodes:
        fig.add_trace(go.Scatter3d(
            x=[n['alpha'] for n in harmonic_nodes],
            y=[n['beta'] for n in harmonic_nodes],
            z=[n['gamma'] for n in harmonic_nodes],
            mode='markers',
            marker=dict(size=6, color='white', symbol='circle', line=dict(color='black', width=1)),
            name='Harmonic Nodes',
            hovertemplate='<b>Node %{text}</b><br>α=%{x:.4f}<br>β=%{y:.4f}<br>γ=%{z:.4f}<br>D=%{customdata:.3f}<extra></extra>',
            text=[str(i+1) for i in range(len(harmonic_nodes))],
            customdata=[n['dissonance'] for n in harmonic_nodes]
        ))

    # Add 12-TET chords (colored by dissonance)
    df_12 = df_chords[df_chords['system'] == '12-TET']
    if len(df_12) > 0:
        fig.add_trace(go.Scatter3d(
            x=df_12['alpha'], y=df_12['beta'], z=df_12['gamma'],
            mode='markers+text',
            marker=dict(
                size=10,
                color=df_12['dissonance'],
                colorscale=custom_colorscale,
                cmin=vmin, cmax=vmax,
                symbol='square',
                line=dict(color='black', width=2),
                showscale=False
            ),
            text=df_12['name'], textposition='top center', textfont=dict(size=10, color='white'),
            name='12-TET Chords',
            hovertemplate='<b>%{text}</b> (12-TET)<br>α=%{x:.4f}<br>β=%{y:.4f}<br>γ=%{z:.4f}<br>D=%{customdata:.3f}<extra></extra>',
            customdata=df_12['dissonance']
        ))

    # Add 53-TET chords (colored by dissonance)
    df_53 = df_chords[df_chords['system'] == '53-TET']
    if len(df_53) > 0:
        fig.add_trace(go.Scatter3d(
            x=df_53['alpha'], y=df_53['beta'], z=df_53['gamma'],
            mode='markers',
            marker=dict(
                size=5,
                color=df_53['dissonance'],
                colorscale=custom_colorscale,
                cmin=vmin, cmax=vmax,
                symbol='circle',
                opacity=1,
                showscale=False
            ),
            name='53-TET Chords',
            hovertemplate='<b>%{text}</b> (53-TET)<br>α=%{x:.4f}<br>β=%{y:.4f}<br>γ=%{z:.4f}<br>D=%{customdata:.3f}<extra></extra>',
            text=df_53['name'], customdata=df_53['dissonance']
        ))

    # Layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(title=dict(text='α (2nd frequency ratio)', font=dict(size=12, color='black')),
                      backgroundcolor='rgb(255, 255, 255)', gridcolor='rgb(230, 230, 230)',
                      tickfont=dict(size=10, color='black'), range=[1.0, 2.0]),
            yaxis=dict(title=dict(text='β (3rd frequency ratio)', font=dict(size=12, color='black')),
                      backgroundcolor='rgb(255, 255, 255)', gridcolor='rgb(230, 230, 230)',
                      tickfont=dict(size=10, color='black'), range=[1.0, 2.0]),
            zaxis=dict(title=dict(text='γ (4th frequency ratio)', font=dict(size=12, color='black')),
                      backgroundcolor='rgb(255, 255, 255)', gridcolor='rgb(230, 230, 230)',
                      tickfont=dict(size=10, color='black'), range=[1.0, 2.0]),
            camera=dict(eye=dict(x=1.0, y=1.0, z=1.0)),
            bgcolor='rgba(255, 255, 255, 1.0)',
            aspectmode='cube'
        ),
        width=800, height=400,
        paper_bgcolor='rgb(255, 255, 255)',
        showlegend=True,
        legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'),
        margin=dict(l=0, r=0, t=50, b=0),
        title=dict(text='EigenSpace: 53-TET vs 12-TET Chord Positions (α ≤ β ≤ γ)', font=dict(size=16, color='black'))
    )

    return fig


# Create and show visualization
print("Generating FIXED visualization...")
fig = create_eigenspace_visualization_fixed(
    alpha_range, beta_range, gamma_range, dissonance_3d,
    harmonic_nodes, df_all_chords,
    show_all_layers=True
)
# fig.show()
print("✓ Visualization complete")

## Export Reference Data

Save the chord mappings and harmonic nodes for use in the data augmentation pipeline.

In [12]:
# ============================================================================
# EXPORT REFERENCE DATA
# ============================================================================

def export_eigenspace_data(
    df_chords: pd.DataFrame,
    harmonic_nodes: List[Dict],
    output_dir: str = "../dataset/EigenSpace_Data"
) -> None:
    """
    Export chord mappings and harmonic nodes to JSON for pipeline use.
    """
    # Create output directory if needed
    os.makedirs(output_dir, exist_ok=True)
    
    # Export chord positions
    chords_export = {
        '12-TET': df_chords[df_chords['system'] == '12-TET'].to_dict('records'),
        '53-TET': df_chords[df_chords['system'] == '53-TET'].to_dict('records')
    }
    
    chords_path = os.path.join(output_dir, "chord_positions.json")
    with open(chords_path, 'w') as f:
        json.dump(chords_export, f, indent=2)
    print(f"✓ Exported chord positions to {chords_path}")
    
    # Export harmonic nodes
    nodes_export = [
        {
            'id': i + 1,
            'alpha': node['alpha'],
            'beta': node['beta'],
            'gamma': node['gamma'],
            'dissonance': node['dissonance'],
            'prominence': node['prominence']
        }
        for i, node in enumerate(harmonic_nodes)
    ]
    
    nodes_path = os.path.join(output_dir, "harmonic_nodes.json")
    with open(nodes_path, 'w') as f:
        json.dump(nodes_export, f, indent=2)
    print(f"✓ Exported {len(nodes_export)} harmonic nodes to {nodes_path}")
    
    # Export 53-TET interval definitions (for reference)
    intervals_export = {
        'thirds': {k: {'steps': v, 'ratio': float(r(v)), 'cents': float(1200 * np.log2(r(v)))} 
                   for k, v in THIRDS_53.items()},
        'fifths': {k: {'steps': v, 'ratio': float(r(v)), 'cents': float(1200 * np.log2(r(v)))} 
                   for k, v in FIFTHS_53.items()},
        'sevenths': {k: {'steps': v, 'ratio': float(r(v)), 'cents': float(1200 * np.log2(r(v)))} 
                     for k, v in SEVENTHS_53.items()}
    }
    
    intervals_path = os.path.join(output_dir, "53tet_intervals.json")
    with open(intervals_path, 'w') as f:
        json.dump(intervals_export, f, indent=2)
    print(f"✓ Exported 53-TET interval definitions to {intervals_path}")


# Export the data
export_eigenspace_data(df_all_chords, harmonic_nodes)

## Summary Tables

### Sorted by Dissonance (Most Consonant First)

In [13]:
# ============================================================================
# SUMMARY TABLES
# ============================================================================

# Most consonant 53-TET chords
print("=" * 70)
print("TOP 20 MOST CONSONANT 53-TET CHORDS")
print("=" * 70)
df_sorted = df_53tet.sort_values('dissonance').head(20)
for i, row in df_sorted.iterrows():
    print(f"{row['name']:15s}  D={row['dissonance']:.4f}  "
          f"α={row['alpha']:.4f} β={row['beta']:.4f} γ={row['gamma']:.4f}")

print("\n" + "=" * 70)
print("MOST DISSONANT 53-TET CHORDS")
print("=" * 70)
df_sorted = df_53tet.sort_values('dissonance', ascending=False).head(10)
for i, row in df_sorted.iterrows():
    print(f"{row['name']:15s}  D={row['dissonance']:.4f}  "
          f"α={row['alpha']:.4f} β={row['beta']:.4f} γ={row['gamma']:.4f}")

## Distance to Nearest Harmonic Node

For each chord, find the nearest local minimum (harmonic node) in EigenSpace.

In [14]:
# ============================================================================
# DISTANCE TO NEAREST HARMONIC NODE
# ============================================================================

def find_nearest_node(
    alpha: float, beta: float, gamma: float,
    nodes: List[Dict]
) -> Tuple[int, float]:
    """
    Find the nearest harmonic node to a given (α, β, γ) position.
    
    Returns:
        (node_id, distance)
    """
    min_dist = float('inf')
    nearest_id = -1
    
    for i, node in enumerate(nodes):
        dist = np.sqrt(
            (alpha - node['alpha']) ** 2 +
            (beta - node['beta']) ** 2 +
            (gamma - node['gamma']) ** 2
        )
        if dist < min_dist:
            min_dist = dist
            nearest_id = i + 1  # 1-indexed
    
    return nearest_id, min_dist


# Calculate nearest node for all chords
nearest_nodes = []
for _, row in df_all_chords.iterrows():
    node_id, distance = find_nearest_node(
        row['alpha'], row['beta'], row['gamma'], harmonic_nodes
    )
    nearest_nodes.append({'nearest_node': node_id, 'node_distance': distance})

df_all_chords = pd.concat([
    df_all_chords.reset_index(drop=True),
    pd.DataFrame(nearest_nodes)
], axis=1)

# Show chords closest to harmonic nodes
print("Chords CLOSEST to Harmonic Nodes:")
print("=" * 70)
df_close = df_all_chords.sort_values('node_distance').head(15)
for _, row in df_close.iterrows():
    print(f"{row['name']:15s} ({row['system']:6s})  → Node {row['nearest_node']:2d}  "
          f"dist={row['node_distance']:.4f}  D={row['dissonance']:.4f}")

print("\n")
print("Chords FARTHEST from Harmonic Nodes:")
print("=" * 70)
df_far = df_all_chords.sort_values('node_distance', ascending=False).head(10)
for _, row in df_far.iterrows():
    print(f"{row['name']:15s} ({row['system']:6s})  → Node {row['nearest_node']:2d}  "
          f"dist={row['node_distance']:.4f}  D={row['dissonance']:.4f}")

## 12-TET to EigenSpace Converter

Function to convert any 12-TET chord (by name) to its EigenSpace coordinates.
This will be used in the augmentation pipeline to find microtonal alternatives.

In [15]:
# ============================================================================
# 12-TET CHORD NAME TO EIGENSPACE CONVERTER
# ============================================================================

# Build lookup table for standard chord types
CHORD_INTERVAL_MAP = {
    # Major family
    'maj7':     (4, 7, 11),   # M3, P5, M7
    'maj':      (4, 7, 11),   # alias
    '7':        (4, 7, 10),   # M3, P5, m7 (dominant)
    'dom7':     (4, 7, 10),   # alias
    '6':        (4, 7, 9),    # M3, P5, M6
    
    # Minor family
    'min7':     (3, 7, 10),   # m3, P5, m7
    'm7':       (3, 7, 10),   # alias
    'minMaj7':  (3, 7, 11),   # m3, P5, M7
    'mM7':      (3, 7, 11),   # alias
    'min6':     (3, 7, 9),    # m3, P5, M6
    'm6':       (3, 7, 9),    # alias
    
    # Diminished family
    'dim7':     (3, 6, 9),    # m3, d5, d7
    'o7':       (3, 6, 9),    # alias
    'hdim7':    (3, 6, 10),   # m3, d5, m7
    'm7b5':     (3, 6, 10),   # alias
    'ø7':       (3, 6, 10),   # alias
    
    # Augmented family
    'aug7':     (4, 8, 10),   # M3, a5, m7
    '+7':       (4, 8, 10),   # alias
    'augMaj7':  (4, 8, 11),   # M3, a5, M7
    '+M7':      (4, 8, 11),   # alias
    
    # Suspended
    'sus4':     (5, 7, 10),   # P4, P5, m7
    'sus2':     (2, 7, 10),   # M2, P5, m7
    '7sus4':    (5, 7, 10),   # alias
    '7sus2':    (2, 7, 10),   # alias
}


def chord_name_to_eigenspace(chord_type: str) -> Optional[Tuple[float, float, float]]:
    """
    Convert a chord type name to EigenSpace (α, β, γ) coordinates.
    
    Args:
        chord_type: Standard chord name (e.g., 'maj7', 'min7', 'dom7')
        
    Returns:
        (α, β, γ) tuple or None if chord type not found
    """
    # Normalize chord name
    chord_type = chord_type.lower().strip()
    
    # Handle common variations
    if chord_type.startswith('-'):
        chord_type = 'min' + chord_type[1:]
    if chord_type == 'delta' or chord_type == 'Δ':
        chord_type = 'maj7'
    
    intervals = CHORD_INTERVAL_MAP.get(chord_type)
    if intervals is None:
        return None
    
    # Convert semitones to frequency ratios
    alpha = r12(intervals[0])
    beta = r12(intervals[1])
    gamma = r12(intervals[2])
    
    return (alpha, beta, gamma)


def find_microtonal_alternatives(
    chord_type: str,
    df_53tet: pd.DataFrame,
    max_distance: float = 0.05
) -> pd.DataFrame:
    """
    Find 53-TET chords near a given 12-TET chord position.
    
    Args:
        chord_type: Standard chord name
        df_53tet: DataFrame of 53-TET chord positions
        max_distance: Maximum EigenSpace distance to consider
        
    Returns:
        DataFrame of nearby 53-TET alternatives, sorted by distance
    """
    pos = chord_name_to_eigenspace(chord_type)
    if pos is None:
        return pd.DataFrame()
    
    alpha, beta, gamma = pos
    
    # Calculate distances to all 53-TET chords
    distances = np.sqrt(
        (df_53tet['alpha'] - alpha) ** 2 +
        (df_53tet['beta'] - beta) ** 2 +
        (df_53tet['gamma'] - gamma) ** 2
    )
    
    # Filter and sort
    df_result = df_53tet.copy()
    df_result['distance_from_12tet'] = distances
    df_result = df_result[df_result['distance_from_12tet'] <= max_distance]
    df_result = df_result.sort_values('distance_from_12tet')
    
    return df_result


# Test the converter
print("Testing 12-TET → EigenSpace converter:")
print("=" * 60)
for chord in ['maj7', 'min7', 'dom7', 'dim7', 'hdim7']:
    pos = chord_name_to_eigenspace(chord)
    if pos:
        print(f"{chord:8s} → α={pos[0]:.5f}, β={pos[1]:.5f}, γ={pos[2]:.5f}")

print("\n")
print("Finding 53-TET alternatives for 'maj7':")
print("-" * 60)
alternatives = find_microtonal_alternatives('maj7', df_53tet, max_distance=0.1)
for _, row in alternatives.head(5).iterrows():
    print(f"  {row['name']:15s}  dist={row['distance_from_12tet']:.4f}  D={row['dissonance']:.4f}")

## Conclusion

This notebook has established:

1. **53-TET Chord Vocabulary**: 55+ microtonal chord types mapped to EigenSpace
2. **Harmonic Nodes**: 77 local minima identified as consonant reference points
3. **Comparison**: 53-TET chords generally have lower dissonance than 12-TET equivalents
4. **Converter**: Functions to translate between chord names and EigenSpace coordinates

### Next Steps (Stage 3 continuation)
- Use these mappings to create microtonal augmentation rules
- Define substitution strategies (10%, 50%, 100% microtonal levels)
- Integrate with MIDI export pipeline for pitch-bend calculations